In [6]:
import numpy as np
import pandas as pd

# custom libs
import sys
sys.path.append("../..")
from src.libs.transformations import obtain_densities, obtain_lqds, obtain_densities_from_lqd
from src.libs.dynamicFPC import K_dFPC, W_dFPC
from src.libs.forecasting import select_order_ic

In [2]:
# Dados
data_path = "../../data/processed/"
returns_path = ''.join([data_path, 'ibovespa_treated.xlsx'])
df = pd.read_excel(returns_path, index_col="time")
df.head()

,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,...,2025-11-14,2025-11-17,2025-11-18,2025-11-19,2025-11-21,2025-11-24,2025-11-25,2025-11-26,2025-11-27,2025-11-28
time,,,,,,,,,,,,,,,,,,,,,
10:00:00,0.000011,-0.000372,-0.000035,0.007233,-0.001305,-0.000459,0.002388,0.002212,-0.008001,-0.000824,...,-0.000957,-0.002307,-0.003734,-0.005020,-0.001139,-0.000255,0.002339,0.004176,-0.000224,-0.000238
10:05:00,-0.002271,0.003372,0.001716,0.005538,0.000069,0.006429,0.002343,0.001918,-0.007582,-0.000413,...,-0.001522,0.000739,-0.000613,0.000306,-0.002021,-0.000810,0.001557,-0.001565,-0.000193,0.002906
10:10:00,-0.000582,0.002832,-0.000502,0.000739,0.000261,0.001891,0.000775,-0.001172,0.000216,-0.000432,...,0.000360,-0.000105,-0.000159,-0.000362,0.000092,0.001131,0.001170,0.000202,0.000556,0.000475
10:15:00,-0.001997,0.001091,0.000111,-0.000192,0.000407,-0.001392,-0.000477,-0.001526,0.000198,-0.001821,...,0.001121,-0.000284,0.003272,-0.001107,-0.001012,-0.000174,0.001002,0.001815,-0.000168,0.000973
10:20:00,-0.000037,-0.000505,-0.000327,-0.000580,-0.000436,-0.000405,0.001263,0.000676,-0.002667,0.001033,...,0.000449,0.000335,0.000853,0.000981,-0.000057,-0.000096,0.000832,0.001951,0.001060,-0.001004


In [3]:
# Obtém objetos functionais
M=3000 
df_densities_supports, df_densities = obtain_densities(df, M=M)
df_densities.head()

,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,...,2025-11-14,2025-11-17,2025-11-18,2025-11-19,2025-11-21,2025-11-24,2025-11-25,2025-11-26,2025-11-27,2025-11-28
0,26.484908,26.956133,43.192058,125.280911,22.281346,38.967352,16.381520,14.790396,14.755901,68.336475,...,56.055614,22.715586,14.822458,15.843866,32.781883,18.742967,33.634914,25.011205,59.591366,35.178493
1,26.549913,26.973634,43.381074,126.228831,22.328213,39.236235,16.389919,14.873119,14.785411,68.447117,...,56.269242,22.760257,14.822055,15.843451,32.878703,18.745446,33.688952,25.117131,59.739446,35.313673
2,26.614794,26.990503,43.570813,127.181415,22.375076,39.506838,16.398066,14.956030,14.814484,68.557193,...,56.483268,22.804829,14.820845,15.842209,32.975476,18.747281,33.742663,25.223321,59.886899,35.448818
3,26.679553,27.006743,43.761277,128.138661,22.421938,39.779179,16.405963,15.039127,14.843118,68.666702,...,56.697690,22.849304,14.818828,15.840138,33.072200,18.748473,33.796047,25.329778,60.033721,35.583923
4,26.744189,27.022353,43.952468,129.100567,22.468800,40.053275,16.413611,15.122411,14.871311,68.775642,...,56.912509,22.893683,14.816005,15.837239,33.168874,18.749022,33.849104,25.436506,60.179908,35.718987


In [4]:
class mLQDT:
    def __init__(
            self, 
            densities : pd.DataFrame,
            densities_supports : pd.DataFrame
            ):
        
        # Data
        self.Y         = densities
        self.Y_support = densities_supports

        # Metadata
        self.columns  = densities.columns
        self.Y_n_rows = densities.shape[0]
        self.Y_n_cols = densities.shape[1]

        # Results 
        self.lqd_support = None
        self.lqd         = None
        self.c           = None


    def densities_to_lqdensities(self, 
                                 lqd_support=None,
                                 verbose=True):
        lqdSup, df_lqds, c = obtain_lqds(
                                        self.Y_support, 
                                        self.Y,
                                        lqd_sup = lqd_support,
                                        verbose_=verbose)
        
        self.lqd_support = lqdSup
        self.lqd         = df_lqds
        self.c           = c

    def lqdensities_to_densities(self, 
                                 adhoc_lqd_support,
                                 adhoc_lqds,
                                 adhoc_c=None,
                                 verbose=True
                                ):
        
        if adhoc_c is not None:
            if verbose:
                print("'c' was not provided. Using original fitted values.")
            adhoc_c = self.c

        densities_supports, densities = obtain_densities_from_lqd(
                                                            adhoc_lqds,
                                                            adhoc_lqd_support,
                                                            c=adhoc_c,
                                                            verbose = True
                                        )
        
        return [densities_supports, densities]

In [ ]:
bovespa_mLQDT = mLQDT(
                    df_densities,
                    df_densities_supports
                )

bovespa_mLQDT.densities_to_lqdensities()

Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5 - renormalizing now.
Density does not integrate to 1 with tolerance 1e-5

In [ ]:
class dFPC:
    def __init__(self, 
                 Y : pd.DataFrame,
                 Y_support : np.array):
        
        # Data
        self.Y = Y
        self.Y_support
        
        # Metadata
        self.Y_columns = Y.columns
        self.Y_n_rows = Y.shape[0]
        self.Y_n_cols = Y.shape[1]

        #Results
        self.eigenfunctions = None
        self.etahat = None
        self.fitted_curves = None

    def fit_KdFPC(self,
                  lag_maximum,
                  alpha,
                  du,
                  B,
                  p,
                  m,
                  u,
                  select_ncomp=False,
                  dimension=None):
        KdFPC_model = K_dFPC(self.Y.values)
        KdFPC_model.fit(
            lag_max=lag_maximum,
            alpha=alpha,
            du=du,
            B=B,
            p=p,
            m=self.Y_n_rows,
            u=self.Y_support,
            select_ncomp=False,
            dimension=dimension
        )
        
        self.eigenfunctions = KdFPC_model.psihat
        self.etahat = KdFPC_model.etahat   
        self.fitted_curves = KdFPC_model.fitted_values

    def fit_WdFPC()

In [ ]:
W_scores = w_scores.real.T

selected_orders = select_order_ic(W_scores)
chosen_lag = selected_orders['aic']
print(f"Chosen lags: {chosen_lag} \n")
var_res = fit_var(W_scores, nlags=chosen_lag)
print(var_res.summary())

In [8]:
from typing import Dict
from statsmodels.tsa.api import VAR

class forecaster:
    def __init__(
            self, 
            Y : pd.DataFrame
            ):
        self.Y = Y

        self.fitted_model = None
    
    def select_order_ic(self,
        maxlags: int = 10,
        criteria: str = 'bic'
        ) -> Dict[str, int]:
        """
        Select lag order using AIC and BIC from statsmodels VAR.select_order.
        Returns dict with keys 'aic', 'bic', 'hqic' (if available).
        """
        model = VAR(self.Y)
        sel = model.select_order(maxlags)
        # statsmodels returns object with attributes aic, bic, hqic that are integers (lags)
        criteria_dict = {'aic': int(sel.aic), 'bic': int(sel.bic), 'hqic': int(sel.hqic)}

        return criteria_dict[criteria]
    
    def fit_var(self, 
                nlags: int
                ) -> VAR:
        """
        Fit a VAR model and return the fitted results object.
        """
        model = VAR(self.Y)
        res = model.fit(nlags)

        self.fitted_model = res

    def forecast(self,
                 h: int
                ):
        """
        Forecast using a fitted statsmodels VARResults object.
        Returns a numpy array (steps x k).
        """
        model = self.fitted_model
        fc_h  = model.forecast(model.endog[-model.k_ar:], steps=h)

        return fc_h

In [10]:
df_lqds = bovespa_mLQDT.lqd

In [13]:
WdFPC_kwargs = {
    "wavelet": 'db2',
    "N": 3,
    "p": 5,
    "d": 2,
    "nt": M,
}

wavelet = 'db2'
N = 3 # wavelet decomposition level
p = 5 # lags
dimensions = 2

WdFPC_model = W_dFPC(df_lqds.values)
WdFPC_model.fit(**WdFPC_kwargs)

WdFPC_fitted = WdFPC_model.Yhat
w_scores = WdFPC_model.scores

c:\Users\mathe\Projetos\densities4risk\notebooks\development\../..\src\libs\dynamicFPC.py:613: ComplexWarning: Casting complex values to real discards the imaginary part
  H[:, m] = pywt.waverec(coeffs_m, wavelet)
c:\Users\mathe\Projetos\densities4risk\notebooks\development\../..\src\libs\dynamicFPC.py:621: ComplexWarning: Casting complex values to real discards the imaginary part
  Yhat[:, t] = mu_hat + H @ scores[:, t]


In [14]:
KdFPC_kwargs = {
    "lag_max": 5,
    "alpha": 0.10,
    "du": 0.05,
    "B": 1000,
    "p": 5,
    "m": M,
    "u": bovespa_mLQDT.lqd_support,
    "select_ncomp": False,
    "dimension": dimensions
}

KdFPC_model = K_dFPC(df_lqds.values)
KdFPC_model.fit(**KdFPC_kwargs)

KdFPC_fitted = KdFPC_model.fitted_values
k_scores = KdFPC_model.etahat